In [1]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 8.6 MB/s 
     |████████████████████████████████| 101 kB 7.8 MB/s 
     |████████████████████████████████| 6.6 MB 37.7 MB/s 


In [2]:
! pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 6.7 MB/s 
     |████████████████████████████████| 212 kB 66.1 MB/s 
     |████████████████████████████████| 115 kB 63.0 MB/s 
     |████████████████████████████████| 127 kB 62.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
!pip install audiomentations

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 71 kB 4.7 MB/s 


# Importando bibliotecas

In [3]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Audio
from scipy.io import wavfile

In [4]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [5]:
from datasets import load_dataset, load_metric

# Puxa dataset

In [6]:
from datasets import load_dataset, Audio

In [7]:
import pandas as pd

In [8]:
import numpy as np


# Rodando Classificador na Original

In [9]:
import pickle
label2id = pickle.load(open("/content/drive/My Drive/Colab Notebooks/audio_ml/datasets/superb_ks_label2id.pkl", "rb"))
id2label = pickle.load(open("/content/drive/My Drive/Colab Notebooks/audio_ml/datasets/superb_ks_id2label.pkl", "rb"))

In [10]:
import pandas as pd
train = pd.read_pickle('/content/drive/My Drive/Colab Notebooks/audio_ml/datasets/superb_ks_original.pkl')

In [11]:
test = pd.read_pickle('/content/drive/My Drive/Colab Notebooks/audio_ml/datasets/superb_ks_test.pkl')

In [12]:
val = pd.read_pickle('/content/drive/My Drive/Colab Notebooks/audio_ml/datasets/superb_ks_validation.pkl')

In [13]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:369: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


In [ ]:
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift

# augment = Compose([
#     AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5),
#     TimeStretch(min_rate=0.8, max_rate=1.25, p=0.5),
#     PitchShift(min_semitones=-4, max_semitones=4, p=0.5),
#     Shift(min_fraction=-0.5, max_fraction=0.5, p=0.5),
# ])

#augment = AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5)

In [ ]:
train.audio.iloc[0]

{'array': array([-0.00115967, -0.00195312, -0.00164795, ..., -0.00317383,
        -0.00366211, -0.00332642], dtype=float32),
 'path': '/root/.cache/huggingface/datasets/downloads/extracted/05734a36d88019a09725c20cc024e1c4e7982e37d7d55c0c1ca1742ea1cdd47f/up/b52bd596_nohash_0.wav',
 'sampling_rate': 16000}

In [ ]:
import random
import numpy as np
def add_noise(audio_dict, n_aug):
  resp = audio_dict.copy()
  audio = resp['array'].astype(np.float32)
  sr = resp['sampling_rate']

  res = []

  for i in range(n_aug):
    minimo = random.uniform(0.0001, 0.001)
    maximo = random.uniform(0.01, 0.025)
    augment = AddGaussianNoise(min_amplitude=minimo, max_amplitude=maximo, p=1)
    perturbed = augment(samples=audio, sample_rate=sr)
    resp['array'] = np.array(perturbed).astype(np.float32)
    res.append(resp)
  return res




In [ ]:
from tqdm.auto import tqdm
tqdm.pandas()
minds_aug = train.copy()

minds_aug['audio_aug'] = minds_aug['audio'].progress_apply(lambda x:add_noise( x, 10))

  0%|          | 0/5110 [00:00<?, ?it/s]

In [ ]:
minds_aug= minds_aug.explode('audio_aug').reset_index(drop=True)
minds_aug.shape

(51100, 4)

In [ ]:
minds_aug.head()

,file,audio,label,audio_aug
0,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,2,{'path': '/root/.cache/huggingface/datasets/do...
1,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,2,{'path': '/root/.cache/huggingface/datasets/do...
2,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,2,{'path': '/root/.cache/huggingface/datasets/do...
3,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,2,{'path': '/root/.cache/huggingface/datasets/do...
4,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,2,{'path': '/root/.cache/huggingface/datasets/do...


In [ ]:
minds_aug_fim = minds_aug.drop('audio', axis =1)
minds_aug_fim.shape

(51100, 3)

In [ ]:
minds_aug_fim.rename(columns={'audio_aug':'audio'},inplace=True)

train = train.append([minds_aug_fim])
train.shape

(56210, 3)

In [ ]:
train.to_pickle('/content/drive/My Drive/Colab Notebooks/audio_ml/datasets/superb_ks_add_noise.pkl')

In [14]:
train = pd.read_pickle('/content/drive/My Drive/Colab Notebooks/audio_ml/datasets/superb_ks_add_noise.pkl')
train = train.sample(frac=1).copy()

In [15]:
train.shape

(56210, 3)

In [16]:
def convert_np(audio_dict):
  resp = audio_dict.copy()
  resp['array'] = np.array(resp['array']).astype(np.float32)
  return resp

In [17]:
train['audio'] = train['audio'].apply(lambda x:convert_np(x))

In [18]:
from datasets import Dataset
train = Dataset.from_pandas(train)
test = Dataset.from_pandas(test)
val = Dataset.from_pandas(val)

In [19]:
max_duration = 1.0
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, 
        sampling_rate=feature_extractor.sampling_rate, 
        max_length=int(feature_extractor.sampling_rate * max_duration), 
        truncation=True, 
    )
    return inputs

In [20]:
encoded_train = train.map(preprocess_function, remove_columns=["audio", "file"], batched=True)

encoded_test = test.map(preprocess_function, remove_columns=["audio", "file"], batched=True)

encoded_val = val.map(preprocess_function, remove_columns=["audio", "file"], batched=True)

  0%|          | 0/57 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:168: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/7 [00:00<?, ?ba/s]

In [21]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    "facebook/wav2vec2-base", num_labels=num_labels, label2id=label2id, id2label=id2label
)

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:369: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForSequenceClassification: ['quantizer.weight_proj.bias', 'project_hid.weight', 'project_hid.bias', 'project_q.bias', 'project_q.weight', 'quantizer.codevectors', 'quantizer.weight_proj.weight']
- This IS expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.weight', 'projector.bias', 'projector.

In [22]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="no",
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=32,
    learning_rate=3e-5,
    warmup_ratio=0.1,
    logging_steps=1,
    num_train_epochs=3,
    metric_for_best_model="accuracy",
)



In [23]:
def compute_metrics(eval_pred):
    acc = load_metric("accuracy")
    f1 = load_metric("f1", average='macro')
    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = acc.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = f1.compute(predictions=predictions, references=labels, average="macro")["f1"]
    return {"accuracy": accuracy, "f1": f1}

In [24]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_train,
    eval_dataset=encoded_val,
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics
)



In [25]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `Wav2Vec2ForSequenceClassification.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `Wav2Vec2ForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 56210
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 4
  Total optimization steps = 1317


Epoch,Training Loss,Validation Loss,Accuracy,F1
0,1.117500,1.092565,0.648573,0.115136
1,0.700200,0.788833,0.747132,0.381556
2,0.665600,0.641752,0.823919,0.559772


***** Running Evaluation *****
  Num examples = 6798
  Batch size = 32


***** Running Evaluation *****
  Num examples = 6798
  Batch size = 32
***** Running Evaluation *****
  Num examples = 6798
  Batch size = 32


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1317, training_loss=1.053193594324109, metrics={'train_runtime': 3084.9517, 'train_samples_per_second': 54.662, 'train_steps_per_second': 0.427, 'total_flos': 1.53080800664832e+18, 'train_loss': 1.053193594324109, 'epoch': 3.0})

In [26]:
predictions = trainer.predict(encoded_test)
print(predictions.predictions.shape, predictions.label_ids.shape)

***** Running Prediction *****
  Num examples = 3081
  Batch size = 32


(3081, 12) (3081,)


In [27]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

In [28]:
acc = load_metric("accuracy")
f1 = load_metric("f1", average='macro')



In [29]:
accuracy = acc.compute(predictions=preds, references=predictions.label_ids)["accuracy"]
accuracy

0.557935735150925

In [30]:
f1 = f1.compute(predictions=preds, references=predictions.label_ids, average="macro")["f1"]
f1

0.4783970818224607